### Model Learning an Image with Taylor polynomial feature map and Dense Neural network.

In [27]:
from tensorflow.keras.models import Model
from keras.models import Sequential
from keras.layers import Dense,  Flatten

import numpy as np
import matplotlib.pyplot as plt
plt.style.use('dark_background')
import cv2
import os
from tensorflow.keras.callbacks import Callback

In [36]:

image_path = "image.jpg" 
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE) 
image_resized = cv2.resize(image, (150, 150))  # adjust you image size and then change everywhere accordingly
image_normalized = image_resized / 255.0  # Normalize pixel values to [0, 1]


model = Sequential([
    # Flatten the input
    Flatten(input_shape=(150, 150, 4)), 
    Dense(16, activation='relu'),
    Dense(16, activation='relu'),
    Dense(150*150, activation='sigmoid'),
    
])
model.summary() 

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_7 (Flatten)             │ (None, 90000)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_21 (Dense)                │ (None, 16)             │     1,440,016 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 16)             │           272 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 22500)          │       382,500 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,822,788 (6.95 MB)

 Trainable params: 1,822,788 (6.95 MB)

 Non-trainable params: 0 (0.00 B)

In [37]:
# defining the taylor function:

def taylor_series_features(input_array, degree):
    taylor_features = []
    for x in input_array:
        features = [x ** i for i in range(degree + 1)]
        taylor_features.append(features)
    return np.array(taylor_features)


In [38]:
# passing the input feature vector to the taylor function to get "Taylor Feature Map"

image_flattened = image_normalized.flatten()
degree = 3
taylor_features = taylor_series_features(image_flattened, degree)
taylor_feature_map = taylor_features.reshape(image_resized.shape[0], image_resized.shape[1], degree + 1)

print(taylor_feature_map.shape)
taylor_feature_map[:2]


(150, 150, 4)


array([[[1.00000000e+00, 8.23529412e-02, 6.78200692e-03, 5.58518217e-04],
        [1.00000000e+00, 4.70588235e-02, 2.21453287e-03, 1.04213312e-04],
        [1.00000000e+00, 4.70588235e-02, 2.21453287e-03, 1.04213312e-04],
        ...,
        [1.00000000e+00, 6.31372549e-01, 3.98631296e-01, 2.51684857e-01],
        [1.00000000e+00, 6.19607843e-01, 3.83913879e-01, 2.37876051e-01],
        [1.00000000e+00, 6.11764706e-01, 3.74256055e-01, 2.28956646e-01]],

       [[1.00000000e+00, 4.31372549e-02, 1.86082276e-03, 8.02707857e-05],
        [1.00000000e+00, 3.13725490e-02, 9.84236832e-04, 3.08780183e-05],
        [1.00000000e+00, 2.35294118e-02, 5.53633218e-04, 1.30266640e-05],
        ...,
        [1.00000000e+00, 6.43137255e-01, 4.13625529e-01, 2.66017987e-01],
        [1.00000000e+00, 6.27450980e-01, 3.93694733e-01, 2.47024146e-01],
        [1.00000000e+00, 6.15686275e-01, 3.79069589e-01, 2.33387943e-01]]])

In [39]:
taylor_feature_map = taylor_features.reshape(150, 150, degree + 1)
taylor_image_bw = np.expand_dims(taylor_feature_map, axis=0)


In [40]:

class SaveReconstructedImageCallback(Callback):
    def __init__(self, model_input, original_shape, output_dir="taylorFm_learned_images", background_color='black'):
        super().__init__()
        self.model_input = model_input
        self.original_shape = original_shape
        self.output_dir = output_dir
        self.background_color = background_color
        os.makedirs(self.output_dir, exist_ok=True)

    def on_epoch_end(self, epoch, logs=None):
        # Predict the reconstructed image
        reconstructed = self.model.predict(self.model_input)
        reconstructed_image = reconstructed.reshape(self.original_shape)

        # Save the reconstructed image
        reconstructed_image = np.clip(reconstructed_image, 0, 1)

        img_name = (f"0{epoch}" if epoch<10 else f"{epoch}")
        save_path = os.path.join(self.output_dir, f'epoch_taylor_{img_name}.png')


        fig, ax = plt.subplots(facecolor=self.background_color) 
        ax.imshow(reconstructed_image, cmap='gray')
        # ax.axis('off') 

        title = f"Taylored FM + NN, Epoch {epoch+1}, Loss: {logs['loss']:.4f}"
        ax.set_title(title)

        # Save the figure
        plt.savefig(save_path, bbox_inches='tight') 
        plt.close()

        print(f"taylor feature map-> learned image saved at epoch {epoch + 1}")

In [41]:

X_train = taylor_image_bw   
Y_train = image_normalized.flatten()  
original_shape = (150, 150)  

save_callback = SaveReconstructedImageCallback(model_input=X_train, original_shape=original_shape)


model.compile(optimizer='adam', loss='mse')
model.fit(X_train, np.expand_dims(Y_train, axis=0), epochs=40, batch_size=1, callbacks=[save_callback])

print("Training complete. Reconstructed images saved in:", save_callback.output_dir)

Epoch 1/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step loss: 0.071
taylor feature map-> learned image saved at epoch 1
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0717
Epoch 2/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - loss: 0.083
taylor feature map-> learned image saved at epoch 2
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 427ms/step - loss: 0.0833
Epoch 3/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - loss: 0.070
taylor feature map-> learned image saved at epoch 3
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 355ms/step - loss: 0.0701
Epoch 4/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.068
taylor feature map-> learned image saved at epoch 4
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.0688  
Epoch 5/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - loss: 0.062
taylor feature map-> learned image saved at epoch 5
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 352ms/step - loss: 0.0629
Epoch 6/40
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - loss: 0.053
taylor feature map-> learned image saved at epoch 6
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 367ms/step - loss:

In [42]:
# Create a video from the saved images
output_dir="taylorFm_learned_images"

def create_video_from_images(image_dir, video_name, fps=2):
    images = sorted([img for img in os.listdir(image_dir)])
    frame = cv2.imread(os.path.join(image_dir, images[0]))
    height, width, layers = frame.shape
    video = cv2.VideoWriter(video_name, cv2.VideoWriter_fourcc(*"mp4v"), fps, (width, height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_dir, image)))

    cv2.destroyAllWindows()
    video.release()

# Generate video
video_name = f"learning_taylor_image.mp4"
create_video_from_images(output_dir, video_name, fps=2)

print(f"Video saved as '{video_name}'")


Video saved as 'learning_taylor_image.mp4'
